In [1]:
import pandas as pd
from sqlalchemy import create_engine
from string import digits
import numpy as np
import os
import nltk
import string
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize  
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williammdavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
csv = "./df.csv"
df = pd.read_csv(csv, encoding = 'unicode_escape')
df.head()


,Index_No,Description_and_Skill,Job_Title,Job_Type,Queried_Salary,<80000,80000-99999,100000-119999,120000-139999,140000-159999,>160000,Salary_Index,Sal_Ind_Cat4-5-6,Skill,No_of_Skills,Location
0,0,"[POSITION SUMMARY, The Business Analyst role i...",Data Scientist,Data Scientist,<80000,1,0,0,0,0,0,1,0,"['SAP', 'SQL']",2.0,MO
1,1,"[What do we need?, You to have an amazing pers...",Data Scientist,Data Scientist,<80000,1,0,0,0,0,0,1,0,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5.0,TX
2,2,"[Validate, analyze, and conduct statistical an...",Data Scientist,Data Scientist,<80000,1,0,0,0,0,0,1,0,"['Data Mining', 'Data Management', 'R', 'SAS',...",9.0,OR
3,3,"[Full time, Washington, DC metro area, Startin...",Graduate Studies Program - Data Scientist,Data Scientist,<80000,1,0,0,0,0,0,1,0,['Certified Internal Auditor'],1.0,DC
4,4,[Assist in consultations with business partner...,Data Scientist I,Data Scientist,<80000,1,0,0,0,0,0,1,0,"['Statistical Software', 'Time Management', 'R...",7.0,TX


In [3]:
df_sample = df.sample(frac =.1) 
df_sample.head()
print(len(df_sample))

572


In [4]:
df3 = df_sample[['Job_Type', 'Description_and_Skill']]
df3.head()

,Job_Type,Description_and_Skill
3935,Data Analyst,[This role will be a part of the Data Reposito...
4417,Data Engineer,"['Java', 'Spring', 'Predictive Analytics', 'S3..."
622,Data Scientist,[The Associate Data Scientist performs data an...
2261,Data Scientist,[US citizenship plus current TS/ SCI + FS Poly...
183,Data Scientist,"[Req ID: 40161, Lubrizol, a Berkshire Hathaway..."


In [5]:
# df3.to_csv('df3.csv')
# df.describe()
# df.dtypes
# len(df3['Description'])

In [6]:
df3.dropna(inplace=True)
len(df3['Description_and_Skill'])

/Users/williammdavis/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


570

In [7]:
def process_text(text):
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean_words

In [8]:
df['Description_and_Skill'].head().apply(process_text)

0    [POSITION, SUMMARY, Business, Analyst, role, p...
1    [need, amazing, personality, communication, st...
2    [Validate, analyze, conduct, statistical, anal...
3    [Full, time, Washington, DC, metro, area, Star...
4    [Assist, consultations, business, partners, in...
Name: Description_and_Skill, dtype: object

In [9]:
#convert collection of text to a matrix of tokens
from sklearn.feature_extraction.text import CountVectorizer
count_v  = CountVectorizer(analyzer=process_text)
message_bow = count_v.fit_transform(df3['Description_and_Skill'])

In [10]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(message_bow, df3['Job_Type'],test_size=0.20, random_state=0)

In [11]:
# message_bow.shape

In [12]:
#create and train the naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
classifier =  MultinomialNB().fit(X_train, y_train)

In [13]:
#print the prediction
print(classifier.predict(X_train))
#print values
print(y_train.values)

['Data Analyst' 'Data Scientist' 'Data Engineer' 'Data Scientist'
 'Data Scientist' 'Data Scientist' 'Data Analyst' 'Data Engineer'
 'Data Engineer' 'Data Engineer' 'Data Engineer' 'Data Engineer'
 'Data Analyst' 'Data Scientist' 'Data Scientist' 'Data Analyst'
 'Data Scientist' 'Data Analyst' 'Data Scientist' 'Data Engineer'
 'Data Analyst' 'Data Analyst' 'Data Scientist' 'Data Analyst'
 'Data Analyst' 'Data Analyst' 'Data Scientist' 'Data Analyst'
 'Data Analyst' 'Data Scientist' 'Data Scientist' 'Data Engineer'
 'Data Engineer' 'Data Analyst' 'Data Analyst' 'Data Analyst'
 'Data Scientist' 'Data Scientist' 'Data Engineer' 'Data Scientist'
 'Data Scientist' 'Data Analyst' 'Data Scientist' 'Data Analyst'
 'Data Scientist' 'Data Scientist' 'Data Analyst' 'Data Engineer'
 'Data Engineer' 'Data Scientist' 'Data Analyst' 'Data Scientist'
 'Data Scientist' 'Data Scientist' 'Data Engineer' 'Data Analyst'
 'Data Scientist' 'Data Scientist' 'Data Analyst' 'Data Engineer'
 'Data Engineer' 'Dat

In [14]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_train)
print(classification_report(y_train, prediction))
print()

                precision    recall  f1-score   support

  Data Analyst       0.95      0.90      0.92       149
 Data Engineer       0.94      0.98      0.96       112
Data Scientist       0.93      0.94      0.94       195

      accuracy                           0.94       456
     macro avg       0.94      0.94      0.94       456
  weighted avg       0.94      0.94      0.94       456




In [15]:
print('Confusion Matrix: \n', confusion_matrix(y_train,prediction))
print('Accuracy Matrix:',  accuracy_score(y_train,prediction))

Confusion Matrix: 
 [[134   2  13]
 [  1 110   1]
 [  6   5 184]]
Accuracy Matrix: 0.9385964912280702


In [16]:
#print the prediction
print(classifier.predict(X_test))
#print values
print(y_test.values)


['Data Analyst' 'Data Scientist' 'Data Scientist' 'Data Engineer'
 'Data Analyst' 'Data Engineer' 'Data Engineer' 'Data Analyst'
 'Data Analyst' 'Data Engineer' 'Data Scientist' 'Data Scientist'
 'Data Engineer' 'Data Scientist' 'Data Scientist' 'Data Analyst'
 'Data Engineer' 'Data Scientist' 'Data Engineer' 'Data Scientist'
 'Data Engineer' 'Data Scientist' 'Data Analyst' 'Data Scientist'
 'Data Scientist' 'Data Analyst' 'Data Engineer' 'Data Scientist'
 'Data Scientist' 'Data Scientist' 'Data Scientist' 'Data Engineer'
 'Data Analyst' 'Data Scientist' 'Data Analyst' 'Data Analyst'
 'Data Scientist' 'Data Analyst' 'Data Scientist' 'Data Analyst'
 'Data Scientist' 'Data Analyst' 'Data Scientist' 'Data Scientist'
 'Data Analyst' 'Data Scientist' 'Data Scientist' 'Data Scientist'
 'Data Analyst' 'Data Scientist' 'Data Analyst' 'Data Analyst'
 'Data Engineer' 'Data Scientist' 'Data Analyst' 'Data Analyst'
 'Data Scientist' 'Data Scientist' 'Data Engineer' 'Data Scientist'
 'Data Analyst'

In [17]:
#Evaluate the model on the training data set 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
prediction = classifier.predict(X_test)
print(classification_report(y_test, prediction))
print()
print('Confusion Matrix: \n', confusion_matrix(y_test,prediction))
print()
print('Accuracy Matrix:',  accuracy_score(y_test,prediction))

                precision    recall  f1-score   support

  Data Analyst       0.73      0.75      0.74        36
 Data Engineer       0.86      0.66      0.75        29
Data Scientist       0.80      0.90      0.85        49

      accuracy                           0.79       114
     macro avg       0.80      0.77      0.78       114
  weighted avg       0.79      0.79      0.79       114


Confusion Matrix: 
 [[27  1  8]
 [ 7 19  3]
 [ 3  2 44]]

Accuracy Matrix: 0.7894736842105263


In [18]:
# print(X_train)

In [19]:
example = ['Weldon Schmidt \
59222 Yundt Falls, Philadelphia, PA \
+1 (555) 408 5416 Work Experience \
07/2017 - PRESENT \
Lead Big Data Engineer \
Boston, MA \
Support, maintain, and document Hadoop and MySQL data warehouse \
Iterate and improve existing features in the pipeline as well as add new ones \
Design, develop, document, and test new requirements in the data pipeline using PERL, BASH, PIG and OOZIE in the Hadoop ecosystem \
Provide full operational support – analyze code to identify root causes of production issues and provide solutions or workarounds and lead it to resolution \
Participate in full development life cycle including requirements analysis, design, development, deployment and operations support \
Work with engineering team members to explore and create interesting solutions while sharing knowledge within the team \
Work across product teams to help solve customer-facing issues \
Demonstrable experience designing technological solutions to complex data problems, developing & testing modular, reusable, efficient and scalable code to implement those solutions \
Big Data Engineer\
10/2011 - 03/2017 \
Dallas, TX \
Plan, develop, monitor and evolve needed infrastructure in collaboration with Ops partners \
Progressive experience as a Systems/Software Engineer, Application Developer or related occupation \
Provides technical design based on business requirements \
Communicates and coordinates with cross functional teams to ensure business objectives are met \
Develops, tests and deploys software applications/systems using scientific analysis and mathematical models to predict and measure outcome and consequences of design \
Provides guidance and mentoring in technical areas to a team of Developer/Analysts \
Documents and submits status reports to leadership \
Provides guidance on new technologies/methodologies \
Junior Big Data Engineer\
09/2004 - 09/2011 \
Detroit, MI \
Designing, developing and maintaining new generation machine learning based big-data web page categorization, data/IP mining, and malicious site detection systems \
Creating and enhancing tools to analyze and process large quantity of data set \
Utilize your programming skills for efficient and robust implementation \
Work closely with malware research/data science teams to enhance malicious site detection, and machine learning/data mining based big data system \
Solve engineering problems by creating solutions that leverage existing technologies and utilize your technical prowess to design and implement solutions where none exist \
Design, build, improve and maintain a high performance and highly scalable services exposed to our solution and a broader development community \
Work with real-time data processing, messaging, streaming techniques and workflows \
University of Sioux Falls \
1998 - 2003 \
Bachelors Degree in Computer Science \
Professional Skills \
Very strong skills working with very large data sets with a variety of tools; experience in dealing with performance and scaling issues \
Demonstrated excellent planning and organizational skills \
Proven design, coding, testing and debugging skills and experience \
Strong analytical, learning and problem solving skills with personal interest in subjects such as math/statistics, machine learning, AI and analytics \
Self-starting, requiring minimal supervision with strong problem-solving skills \
Relevant software development / consulting experience with strong skill on database modeling and architecture \
Strong execution skills in a fast-paced environment using agile methodologies']
input_bow = count_v.transform(example)
print(input_bow)
# classifier =  MultinomialNB().fit(X1_train, y1_train)
classifier.predict(input_bow)

  (0, 14)	1
  (0, 157)	1
  (0, 266)	1
  (0, 509)	1
  (0, 627)	1
  (0, 657)	1
  (0, 714)	3
  (0, 760)	1
  (0, 1115)	1
  (0, 1156)	1
  (0, 1266)	1
  (0, 1367)	1
  (0, 1373)	3
  (0, 1420)	1
  (0, 1437)	1
  (0, 1439)	1
  (0, 1458)	2
  (0, 1463)	1
  (0, 1481)	1
  (0, 1486)	1
  (0, 1541)	1
  (0, 1721)	2
  (0, 1831)	1
  (0, 1909)	2
  (0, 2252)	2
  :	:
  (0, 11584)	3
  (0, 11613)	1
  (0, 11672)	1
  (0, 11684)	2
  (0, 11721)	1
  (0, 11725)	1
  (0, 11778)	3
  (0, 11794)	3
  (0, 11801)	3
  (0, 11805)	1
  (0, 11811)	1
  (0, 11813)	1
  (0, 11857)	1
  (0, 11861)	2
  (0, 11862)	1
  (0, 11967)	2
  (0, 12236)	3
  (0, 12243)	1
  (0, 12276)	1
  (0, 12398)	1
  (0, 12420)	1
  (0, 12440)	1
  (0, 12494)	1
  (0, 12509)	1
  (0, 12512)	1


array(['Data Scientist'], dtype='<U14')

In [24]:
!pip install nbconvert
!jupyter nbconvert --to python Project_3.ipynb
# !mv mission_to_mars.py scrape_mars.py

You should consider upgrading via the '/Users/williammdavis/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
[NbConvertApp] Converting notebook Project_3.ipynb to python
[NbConvertApp] Writing 6825 bytes to Project_3.py


In [26]:
import pickle


In [28]:
# class NaiveBayesClpickl = {'model': count_v}
pickle.dump( pickl, open( 'model_file' + ".p", "wb" ) )